##Making Friends with Pandas
###A Look at Jon Stewart's Historical Guest Record

I'd like to get more comfortable with manipulating and analyzing structured data in Python, as that is the foundation of pretty much any analysis.  This fun data set will serve as an opportunity to explore the similarities and differences between Pandas and data frames in R.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Thanks to FiveThirtyEight for getting this dataset together
guests = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/data/master/daily-show-guests/daily_show_guests.csv')

There is one notable difference right away.  It is quite straightforward to read a csv directly from the web, even with SSL encryption.  This takes several lines of code in R.

In [3]:
guests.head()

,YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List
0,1999,actor,1/11/99,Acting,Michael J. Fox
1,1999,Comedian,1/12/99,Comedy,Sandra Bernhard
2,1999,television actress,1/13/99,Acting,Tracey Ullman
3,1999,film actress,1/14/99,Acting,Gillian Anderson
4,1999,actor,1/18/99,Acting,David Alan Grier


In [4]:
guests.YEAR.head()

0    1999
1    1999
2    1999
3    1999
4    1999
Name: YEAR, dtype: int64

Columns within the data frame can be accessed using the attribute operator (.).  Now how about slicing off the first year of data, with the Year, Show, Group, and Raw_Guest_List columns.

In [5]:
guests99 = guests[guests.YEAR == 1999]

In [6]:
guests99.shape

(166, 5)

In [7]:
guests99 = guests99.ix[:, [0, 2, 3, 4]] #ix method supports label based and location based indexing

In [8]:
guests99.shape

(166, 4)

Great, these indexing methods seem to be working more or less as I'd expect, however I couldn't figure out how to subset the rows using the boolean expression and also subset the columns with their numerical indices within the same line of code.  Now let's try some split-apply-combine techniques, similar to what you could do with plyr in R.  Let's figure out who the most frequent Daily Show guest was in 1999.

In [9]:
gb = guests99.groupby('Raw_Guest_List', as_index = False)

In [10]:
show_count = gb['Show'].agg({'Shows' : len}) #return a df with counts of 
#shows that each guest has been on. Passing a dict to the agg function 
#allows for choosing a column name if desired.
show_count.head()

,Raw_Guest_List,Shows
0,Adam Arkin,1
1,Adam Sandler,1
2,Amy Brenneman,1
3,Andy Richter,1
4,Angelina Jolie,1


In [11]:
#almost there, now we just need to sort the new data frame to 
#see the most frequent guests
show_count = show_count.sort('Shows', ascending = False)
show_count.head(10)

,Raw_Guest_List,Shows
103,"Pamela Anderson, Natalie Raitano, Molly Culver",3
150,Yasmine Bleeth,2
25,Dave Foley,2
89,Melissa Gilbert,2
95,Mike Myers,2
101,Norm Macdonald,2
55,"Hootie & the Blowfish, Billy Crystal",2
48,George Carlin,2
45,"Frank DeCaro's Oscar Special, John Larroquette",2
115,Rob Schneider,2


At first glance, it would seem that groups of multiple guests returned for multiple shows.  However it seems coincidental that the number of shows equals the number of guests.  Referencing back to the original data confirms that it was compiled such that shows with multiple guests have a line for each guest, even though the entire group is listed in each row.  It is easy enough to look past this and see that there are several guests tied for first with two appearances, but for the sake of this exercise, let's see if we can remove the multiple guests with a regular expression.

In [12]:
pattern = ','
show_count.Raw_Guest_List.str.contains(pattern)

103     True
150    False
25     False
89     False
95     False
101    False
55      True
48     False
45      True
115    False
79     False
30     False
39     False
9      False
139    False
96     False
108    False
107    False
106    False
105    False
104    False
145    False
146    False
102    False
147    False
100    False
99     False
98     False
97     False
148    False
       ...  
66     False
67     False
68     False
70     False
61     False
71     False
72     False
73     False
74     False
1      False
76     False
62     False
60     False
41     False
50     False
42     False
43     False
44     False
46     False
47     False
49     False
51     False
59     False
52     False
53     False
54     False
56     False
57     False
58     False
75     False
Name: Raw_Guest_List, dtype: bool

In [13]:
idx = show_count.Raw_Guest_List.str.contains(pattern)

In [14]:
#it does work to subset the data frame by placing the bool list 
#in brackets, but using [-idx] to subset rows where the .contains 
#function is False does not seem to work.  Perhaps there is a better 
#way to do this, but for now I'll just assign the index as its inverse.
idx = -idx
print idx

103    False
150     True
25      True
89      True
95      True
101     True
55     False
48      True
45     False
115     True
79      True
30      True
39      True
9       True
139     True
96      True
108     True
107     True
106     True
105     True
104     True
145     True
146     True
102     True
147     True
100     True
99      True
98      True
97      True
148     True
       ...  
66      True
67      True
68      True
70      True
61      True
71      True
72      True
73      True
74      True
1       True
76      True
62      True
60      True
41      True
50      True
42      True
43      True
44      True
46      True
47      True
49      True
51      True
59      True
52      True
53      True
54      True
56      True
57      True
58      True
75      True
Name: Raw_Guest_List, dtype: bool


In [15]:
show_count = show_count[idx]
show_count.head(10)

,Raw_Guest_List,Shows
150,Yasmine Bleeth,2
25,Dave Foley,2
89,Melissa Gilbert,2
95,Mike Myers,2
101,Norm Macdonald,2
48,George Carlin,2
115,Rob Schneider,2
79,Kellie Martin,2
30,David Cross,2
39,Eduardo Sanchez and Daniel Myrick,2


Alright, there we have ten of the most frequent guests on the Daily Show in 1999.  How's that for a time capsule?